In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import warnings
from sklearn.exceptions import DataConversionWarning 
import collections
import copy
import json
from math import sqrt
import scipy as sp
import sklearn
import sklearn.preprocessing
from sklearn.utils import check_random_state
from sklearn.externals import joblib
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import GridSearchCV , cross_val_score
from treeinterpreter import treeinterpreter as ti
from functools import partial
import keras
from keras.models import load_model, model_from_json
import keras.backend as K
import tensorflow as tf

warnings.filterwarnings(action = 'ignore',category = DataConversionWarning)
warnings.filterwarnings(action = 'ignore',category = FutureWarning)

Using TensorFlow backend.


**Loading the trained deep learning Model. This model is a binary classifier that predicts if the molecule is toxic or not.**

In [ ]:
def load_mymodel(jsonfile_path, weight_path):
    '''
    Return loaded model
    Input:
    jsonfile_path = path to model achitecture
    weight_path: path to models trained weight
    '''
    json_file = open(jsonfile_path, 'r')
    loaded_model_json = json_file.read()
    p = json.loads(loaded_model_json)
    model = model_from_json(p)
    model.load_weights(weight_path)
    return(model)


def build_masked_loss(loss_function, mask_value):
    """
    Inputs:
    loss_function: The loss function to mask
    mask_value: The value to mask in the targets

    Returns:
    function: a loss function that acts like loss_function with
    masked inputs
    """

    def masked_loss_function(y_true, y_pred):
        mask = K.cast(K.not_equal(y_true, mask_value), K.floatx())

        return loss_function(y_true * mask, y_pred * mask)

    return 0


keras.losses.masked_loss_function = build_masked_loss

**Loading the Model**

In [ ]:
jsonfile_path = "../Final_tutorial/Model_architecture_Epoch14.json"
weight_path = "../Final_tutorial/Model_weights_Epoch14.h5"
Final_model = load_mymodel(jsonfile_path , weight_path)

**Reading the Train and Test Data**

In [ ]:
np.random.seed(1)
# Training Data
train  = pd.read_csv("../Final_tutorial/Ai_train_ToxAlerts_standardised_w_activity.csv")
train = train.drop(["Clusters", "Fold"], axis=1)
inp_train = train.dropna(0)
y_train = inp_train["Activity"]
ids_train = inp_train.Reference
inp_train = inp_train.drop(["Reference", "Activity"], axis=1)
features_name = list(inp_train)
inp_train = inp_train.values

#Test Data
test = pd.read_csv("../Final_tutorial/Ai_test_ToxAlerts_standardised_w_activity_wo_trainingset_molecules.csv")
test = test.dropna(axis=0)
preds = {}
inp = test.dropna(0)
y = inp["Activity"]
ids = inp.Reference
inp = inp.drop(["Reference", "Activity"], axis=1)
inp = inp.values
pred = Final_model.predict(inp)

**Function to get the training distribution and neighbourhood data accordingly**

The probablity of an observation to get sampled as a neighbourhood of a molecule depends on the distance. We assigned weights to each molecule. High weights if the model is close and low weights if it is far

$$w_i = \sqrt{ \dfrac{exp(-d_i)}{wd^2}}$$

where d is the distance between two points.\
wd is a scaling parameter

We used jaccard similarity as a distance metrics
$$Jaccard Similarity = \dfrac{A\cap B}{A\cup B}$$

In [ ]:
def get_training_distribution(training_data):
    """
    We do the sampling of neighbourhood dataset according to training_data distribution. This function returns 
    columnwise fraction of 1s and 0s.
    
    Args:
    
    training_data = numpy array of training data of shape (num_of_obs , num_of_features)
    
    Returns :
    
    feature_values = represents catagories present in every column, for bitwise vectors its always [0,1]
    feature_frequency =represent the list of columnwise fraction of 1s and 0s.Returned in dictionary format where key is column number 
    
    
    """
    feature_values = {}
    feature_frequencies = {}
    for feature in categorical_features:
        column = training_data[:, feature]
        feature_count = collections.Counter(column)
        values, frequencies = map(list, zip(*(sorted(feature_count.items()))))
        feature_values[feature] = values
        feature_frequencies[feature] = np.round((np.array(frequencies) /
                                             float(sum(frequencies)) ) ,decimals=2)
    
    return feature_values , feature_frequencies




def get_neighbourhood_data(data_row,num_samples,feature_values,feature_frequencies,upsampling_parameter):
    '''
    Fits the fine tuned decision tree model on neighbourhood dataset.The final model is then fed into treeinterpreter 
    to calculate the feature importance for the test case
    
    Args : 
    
    neighborhood_data = numpy array of neighbourhood dataset 
    labels_column = labels of neighborhood_data predicted from deep learning model 
    weights = weights according to jaccard distance. Far the molecule from the test case lower the weight and vice versa.
    
    
    Return :
    
    easy_model = The best model after tuning min_samples_leaf
    local_pred = Local prediction for the test case 
    scores = 5 folds cross validation scores
    min_samples_leaf = The best value of min_samples_leaf after cross validation 
    
    '''
    num_cols = data_row.shape[0]
    data = np.zeros((num_samples, num_cols))
    first_row = data_row
    data[0] = data_row.copy()
    inverse = data.copy()

    for column in categorical_features:
        values = feature_values[column]
        freqs = feature_frequencies[column]
        #Few of the freqs have different shape 
        if freqs.shape==(1,):
            freqs = np.asarray([0. ,1. ])
            values = [0.0, 1.0]
        #This part make sure that tweaking is around fixed bid only that is one 
        if first_row[column] ==0:
            freqs = np.asarray([1. ,0. ])
            
        ##THis part will add the fucntionality of oversampling more ones.
        freq_one = freqs[1]
        freq_one=freq_one*(1+upsampling_parameter)
        if freq_one > 1.0 : 
            freq_one = 1.0
        freq_zero = 1.0 - freq_one
        freqs=np.asarray([freq_zero,freq_one])
        #######
        random_state = column
        random_state = check_random_state(random_state)
        inverse_column = random_state.choice(values, size=num_samples,replace=True, p=freqs)

        binary_column = np.array([1 if x == first_row[column]
                                  else 0 for x in inverse_column])
        binary_column[0] = 1
        inverse_column[0] = data[0, column]
        data[:, column] = binary_column
        inverse[:, column] = inverse_column

    inverse[0] = data_row
    neighborhood_data = inverse
    return neighborhood_data  


def calculate_distances(dataset , datapoint):
    '''
    Calculates the jaccard distance between all the observations of dataset to datapoints.This is used to weigh the
    neighbourhood observations according to test case(datapoint for the function)
    
    
    Args : 
    
    Dataset = A numpy array of shape (num_of_obs , num_of_features)
    datapoint = A numpy array of shape (num_of_features, )
    
    Return:
    
    distances = Pairwise jaccard distance 
    '''
    distances = sklearn.metrics.pairwise_distances(
        dataset,
        datapoint.reshape(1, -1),
        metric='jaccard'
    ).ravel()
    
    return distances 



def get_weights_from_distances(distances , scaling_factor):
    '''
    Calculates the weights from the distance. It is used as scaling tool where all the distances are converted between 0-1
    The function that convert that is called kernel function and the formula is sqrt( exp(-distance)/kernel_width ** 2 ).
    
    Args :
    
    distance : Numpy array of distances.Its the output from calculate_distances 
    scaling_factor: to increase the gap between the weights ,should be around 35 for our dataset.
    
    '''
    kernel_width = np.sqrt(training_data.shape[1]) * .75
    kernel_width = float(kernel_width)
    scaled_distance= distances*scaling_factor
    weights=np.sqrt(np.exp(-(scaled_distance ** 2) / kernel_width ** 2)) 
    return weights




def neighbourhood_model(neighborhood_data,labels_column,weights):
    '''
    Fits the fine tuned decision tree model on neighbourhood dataset.The final model is then fed into treeinterpreter 
    to calculate the feature importance for the test case
    
    Inputs : 
    
    neighborhood_data = numpy array of neighbourhood dataset 
    labels_column = labels of neighborhood_data predicted from deep learning model 
    weights = weights according to jaccard distance. Far the molecule from the test case 
             lower the weight and vice versa.
    
    
    Return :
    
    easy_model = The best model after tuning min_samples_leaf
    local_pred = Local prediction for the test case 
    scores = 5 folds cross validation scores
    min_samples_leaf = The best value of min_samples_leaf after cross validation 
    
    
    '''
    
    param_grid = {'min_samples_leaf' :[2,5,10,20,25,30,40,50,60,70] }#
    easy_model = GridSearchCV(DecisionTreeRegressor(random_state= random_state),param_grid,cv = 5)#
    easy_model.fit(neighborhood_data,
                   labels_column, sample_weight=weights)
    
    best_param = easy_model.best_params_ #
    easy_model = DecisionTreeRegressor(random_state= random_state,min_samples_leaf=best_param['min_samples_leaf'])
    easy_model.fit(neighborhood_data,
                   labels_column, sample_weight=weights)
    residual = easy_model.score(
        neighborhood_data,
        labels_column, sample_weight=weights)
    scores = cross_val_score(easy_model, neighborhood_data, labels_column, cv=5)
    local_pred = easy_model.predict(neighborhood_data[0,:].reshape(1, -1))
    y_pred= easy_model.predict(neighborhood_data)
    

    return easy_model,local_pred ,residual ,scores ,best_param['min_samples_leaf']


def get_top3_sorted(contributions , feature_names):
    '''
    This function will sort the features/feature_combinations according to their absolute importance and returns 
    the top 3 important feature/feature_combinations.The features contributions are also scaled by dividing all 
    the contributions with sum of absolute value of contribution 
    
    Args: 
    contributions = Contribution list which is the output from the tree interpreter 
    feature_names = Names of features in training dataset
    
    Returns:
    
    final_dict = Dictionary containing the top 3 important features and their scaled contribution.
    '''
    current =contributions[0]
    sorted_key =sorted(current, key=lambda k: abs(current[k]),reverse=True)
    abs_sum = return_AbsSum(current)
    current = {k: (v / abs_sum) for k, v in current.items()}
    current = {sorted_key[0]:current[sorted_key[0]], sorted_key[1]:current[sorted_key[1]] ,sorted_key[2]:current[sorted_key[2]]}

    #Propertly assigning the feature names according to training dataset 

    final_dict ={}
    for key in current.keys():
        change_key = []
        for element in list(key):
            element = feature_names[element]
            change_key.append(element)
        final_dict[tuple(change_key)] =current[key]

    return final_dict

def absolute_sum(myDict): 
    '''
    Calculates the sum of absolute value of feature_contribution. This is used to properly scale the feature contribution.
    This function is automatically called in get_top3_sorted()  function and doesn't need to be called specifically.
    
    Args:
    
    myDict = Sorted contribution dictionary  
    
    Returns:
    
    sum = return absolute sum of all the contribution.
    '''

    return sum(abs(myDict.values))





**Input to the local model**

With local model we are trying to understand how the decisions were made at local decision boundary. Here data row is the test example for which we are trying to find out the explainations. We need training data to get the distribution of 0s and 1s for each feature which will be further used to generate neighbourhood dataset. `predict_fn` will be used to get the training_data labels

In [ ]:
training_data=inp_train         # numpy array of training dataset without label.
feature_names=features_name     #name of the features in the dataset in the list format
categorical_features=list(range(training_data.shape[1]))
predict_fn = Final_model.predict  #trained deep learning predict function ,should output the probability
data_row=inp[5]       #test case 
num_samples = 5000   #Number of neighbourhood to be generated
upsampling_parameter = 0.0 

**Get feature distribution from the training dataset**

While generating the neighbourhood dataset we will only tweak the features having value == 1 for the test case. So the feature with value ==0 will stay zero for all the generated neighbouhood dataponts and will be of no importance for local level interpretable model 

In [ ]:
random_state = 42
random_state = check_random_state(random_state)
feature_values , feature_frequencies = get_training_distribution(training_data)

In [12]:
#feature_frequencies

**Generate the neighbourhood dataset.**

In [ ]:
neighborhood_data=get_neighbourhood_data(data_row,num_samples,feature_values,feature_frequencies,upsampling_parameter)

In [ ]:
neighborhood_data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

**Calculates the distance of test case from training dataset. Then find that 22% of that maximum distance and call it as r_fidelity**

To check how accurately our interpretable model is capturing the local decision boundary we will check the R2(r-squared) scored in a circle of radius 0.22*r_max ,where r_max is the maximum distance of test case from the training sample.

In [ ]:
distances_train=calculate_distances(training_data , neighborhood_data[0])
r_fidelity =0.22*max(distances_train)
print(r_fidelity)

0.20533333333333334


**Distance of test case from all the generated neighbourhood samples**

We will weigh the neighbourhood datapoints according to the test case,so that to be in local space the points that are closer will be given high weight and vice versa.The following distance calculates the jaccard distance.

In [ ]:
distances = calculate_distances(neighborhood_data , neighborhood_data[0])
print('sorted distances are :', distances)

sorted distances are : [0.         0.66666667 0.66666667 ... 0.68888889 0.68888889 0.64444444]


As you can see the sorted list the distances are very closer so we multiplied the distance with scaling_factor so that the weights are properly separeted.This step should to properly checked.

In [ ]:
scaling_factor = 35
weights=get_weights_from_distances(distances , scaling_factor)

print(weights)
print('sorted weights to check the separation :',np.sort(weights) )

[1.         0.638574   0.638574   ... 0.61945374 0.61945374 0.65762864]
sorted weights to check the separation : [0.54308879 0.54308879 0.54308879 ... 0.8192703  0.8192703  1.        ]


As you can see the weights are now from 0.63 to 0.81 for this case and are well separated

**Get the predictions from the deep learning model**

We will predict the output for neighbourhood dataset using deep learning model and these output will be used as labels for our local model. 

In [ ]:
neighborhood_labels = predict_fn(neighborhood_data)

**Build the interpretable model at the local space**

With generated neighbourhood data as new training data for local model, deep learning prediction for this data as the label and weighed by the distance we built a fine tuned local interpretable model 

In [ ]:
labels_column = neighborhood_labels[:, 0] 
easy_model,local_pred ,residual ,scores ,min_samples_leaf= neighbourhood_model(neighborhood_data,
                                                                               labels_column,weights)

In [ ]:
print('Residual of the best model is:  ', residual)
print('Local prediction of the model is:  ',local_pred)
print('Right prediction of the model is: ',labels_column[0] )
print('Five fold cross validation scores are: ',scores)
print('Best min sample leaf is: ' , min_samples_leaf)

Residual of the best model is:   0.8368967751403357
Local prediction of the model is:   [0.44859488]
Right prediction of the model is:  0.27391064
Five fold cross validation scores are:  [0.61584687 0.61524735 0.59794527 0.63370461 0.61151395]
Best min sample leaf is:  5


**Getting the feature importance through tree interpreter**

tree interpreter represents prediction = `bias + feature_1_contribution +feature_2_contribution+ all possible pairs.` 


So the negative value of feature contribution is trying to pull prediction towards zero and positive value in contribution is trying to push the rediction towards 1

In [ ]:
prediction, bias, contributions = ti.predict(easy_model, neighborhood_data[0,:].reshape(1,-1),joint_contribution=True)

In [ ]:
contributions

[{(6,): array([-0.09581262]),
  (6, 15): array([-0.03536325]),
  (6, 15, 40): array([0.18496529]),
  (6, 15, 40, 177): array([-0.25022517]),
  (6, 15, 40, 71, 177): array([-0.06499414])}]

In [ ]:
final_dict=get_top3_sorted(contributions , feature_names)

In [ ]:
final_dict

{('TA1095', 'TA1301', 'TA390', 'TA9503'): array([-0.39632696]),
 ('TA1095', 'TA1301', 'TA390'): array([0.29296306]),
 ('TA1095',): array([-0.15175581])}